In [163]:
import numpy as np
from io import BytesIO

In [164]:
def normalize(p):
    return p / np.sum(p)

In [165]:
def sinkhorn_barycenter(Lambda, C, p, Gamma, L):

    K = np.exp(- C / Gamma)
    S = len(p)
    N = len(p[0])
    a = np.ones((N, S))
    b = np.ones((N, S))

    for _ in range(L):
        P = np.ones(S)

        for k in range(S):
            P *= (K.T @ a[:, k]) ** Lambda[k]

        for k in range(S):
            b[:, k] = P / (K.T @ a[:, k])

        for k in range(S):
            a[:, k] = p[:, k] / (K @ b[:, k])

    return P

In [166]:
def sinkhorn_differentiate(Lambda, C, p, q, Gamma, L):
    
    K = np.exp(- C / Gamma)
    S = len(p)
    N = len(p[0])
    Phi = np.ones((L+1, N, S))
    b = np.ones((L+1, N, S))
    w = np.zeros(S)
    r = np.zeros((N, S))
    
    for i in range(1, L+1):
        P = np.ones(S)

        for j in range(S):
            Phi[i, :, j] = K.T @ (p[:, j] / (K @ b[i-1, :, j]))

        for j in range(S):
            P *= Phi[i, :, j] ** Lambda[j]

        for j in range(S):
            b[i, :, j] = P / Phi[i, :, j]

    g = (p - q) * P

    for i in range(1, L+1):

        for j in range(S):
            w[j] += np.dot(np.log(Phi[L-i+1, :, j]), g)

        for j in range(S):
            r[:, j] = - K.T @ ((K @ ((Lambda[j] * g - r[:, j]) / Phi[L-i+1, :, j])) * (p[:, j] / (K @ b[L-i, :, j]) ** 2)) * b[L-i, :, j]

        g = np.zeros(N)
        for j in range(S):
            g += r[:, j]

    return P, w

In [167]:
def image_to_array(imag):
    """
    Convert a black-and-white image to a 2D array where
    0 represents white pixels and 1 represents black pixels.
    
    Parameters:
        image_path (str): The path to the image file.
    
    Returns:
        list: A 2D list of 0s and 1s representing the image.
    """
    # Open the image
    img = image_path.convert('L')  # Convert to grayscale
    # Convert image to binary (black and white)
    threshold = 128  # Define a threshold for binarization
    binary_img = img.point(lambda x: 0 if x > threshold else 1, '1')
    
    # Convert the binary image to a numpy array
    array = np.array(binary_img, dtype=int)
    
    # Return the array as a list of lists
    return array

In [178]:
from PIL import Image
img1 = Image.open('Images/ML_SHAPES/1.png')

In [187]:
np.asarray(img1)

array(<module 'PIL.Image' from '/opt/conda/lib/python3.12/site-packages/PIL/Image.py'>,
      dtype=object)

In [190]:
img1 = Image.open('Images/ML_SHAPES/1.png')
img2 = Image.open('Images/ML_SHAPES/2.png')
img3 = Image.open('Images/ML_SHAPES/3.png')

shape1 = normalize(np.asarray(img1).reshape(-1))
shape2 = normalize(np.asarray(img2).reshape(-1))
shape3 = normalize(np.asarray(img3).reshape(-1))

shapes = np.array([shape1, shape2, shape3])

In [192]:
sinkhorn_differentiate(1, 10, shapes, 10, 10)

TypeError: sinkhorn_differentiate() missing 1 required positional argument: 'L'